In [ ]:
import kotlinx.coroutines.Dispatchers
import kotlinx.coroutines.async
import kotlinx.coroutines.awaitAll
import kotlinx.coroutines.runBlocking
import com.google.common.hash.Hashing
import io.github.cdimascio.dotenv.Dotenv
import java.nio.file.Paths

%useLatestDescriptors
%use lets-plot

class Main

val dotenv = Dotenv.load()
val dataDir = dotenv.get("DATA_DIR").let { Paths.get(it).toFile() }.also { it.mkdirs() }
dataDir

In [ ]:
import org.bson.BsonDocument
import org.litote.kmongo.*

val client = KMongo.createClient("mongodb://localhost:42692/")
val db = client.getDatabase("s5_snyk_libio")
val vulnCollection = db.getCollection<BsonDocument>("mergedVuln")

In [ ]:
import com.mongodb.client.*

private class MongoCursorIterable<T>(private val cursor: MongoCursor<T>) : MongoCursor<T> by cursor, Iterable<T> {

    override fun iterator(): Iterator<T> = cursor
}

private fun <T> MongoIterable<T>.kCursor(): MongoCursorIterable<T> = MongoCursorIterable(iterator())

fun <T, R> MongoIterable<T>.useCursor(block: (Iterable<T>) -> R): R {
    return kCursor().use(block)
}

In [ ]:
import org.bson.BsonNull
import org.bson.BsonString
import org.jetbrains.kotlinx.dataframe.math.mean
import org.jetbrains.kotlinx.dataframe.math.median

val vulnGavToClasses = vulnCollection.find().useCursor { blk ->
    blk.map {
        val gav = it["vuln_gav"]!!.asString().value
        val vulnClasses = it["vuln_classes"]!!.asArray().map { it.asString().value }.toSet()

        if (vulnClasses.count() == 0) throw Exception("no vuln class (record should have been ommitted previously)")

        gav to vulnClasses
    }
}.groupBy { it.first }.map { g -> g.key to g.value.flatMap { it -> it.second }.toSet() }.toMap()

println(vulnGavToClasses.count())
println(vulnGavToClasses.map { it.value.size }.minOrNull())
println(vulnGavToClasses.map { it.value.size }.maxOrNull())
println(vulnGavToClasses.map { it.value.size }.average())
println(vulnGavToClasses.map { it.value.size }.median())

In [ ]:
val vulnCveGavToClasses = vulnCollection.find().useCursor { blk ->
    blk.map {
        val gav = it["vuln_gav"]!!.asString().value
        val cve = it["cve_ref"]!!.asString().value.let { c -> if (c.isBlank()) it["snyk_url"]!!.asString().value else c }
        val vulnClasses = it["vuln_classes"]!!.asArray().map { it.asString().value }.toSet()

        if (vulnClasses.count() == 0) throw Exception("no vuln class (record should have been ommitted previously)")

        (cve to gav) to vulnClasses
    }
}.groupBy { it.first }.map { g -> g.key to g.value.flatMap { it -> it.second }.toSet() }.toMap()
    
println(vulnCveGavToClasses.count())
println(vulnCveGavToClasses.map { it.value.size }.minOrNull())
println(vulnCveGavToClasses.map { it.value.size }.maxOrNull())
println(vulnCveGavToClasses.map { it.value.size }.average())
println(vulnCveGavToClasses.map { it.value.size }.median())

In [ ]:
val data = mapOf(
    "x" to vulnCveGavToClasses.map { it.value.size } + vulnGavToClasses.map { it.value.size },
    "t" to vulnCveGavToClasses.map { "vulnGavClassesPair" } + vulnGavToClasses.map { "vulnGavToClasses" }
)

val p = letsPlot(data) +
        geomHistogram(alpha = .3, binWidth = 1, center = 1.5) { x = "x"; color = "t"; fill = "t" } +
        xlim(limits = Pair(1, 10)) +
        ggsize(1800, 800) 
p

In [ ]:
vulnCveGavToClasses.asSequence().shuffled().first()

In [ ]:
val depGraphCacheDir = dataDir.resolve("interim/depGraphCache")

val gavToDepGraphCache = vulnGavToClasses
    .map {
        val cachePath = depGraphCacheDir.resolve("${it.key}.vertices.tsv.zip")
        if (!cachePath.isFile()) throw Exception("cache file not found")
        
        it.key to cachePath
    }
    .toMap()
gavToDepGraphCache.count()

there are 3812 gavs

In [ ]:
import common.DefaultGraph
import io.github.classgraph.ClassInfoList
import java.io.File
import java.net.URLClassLoader
import org.jgrapht.Graph
import org.jgrapht.graph.DefaultEdge
import org.jgrapht.graph.builder.GraphTypeBuilder

fun loadDepGraph(gav: String): DefaultGraph = 
    scripts.exportDepGraphs.loadDepGraphFromCache(gav)

fun loadClassListInfo(gav: String): ClassInfoList =
    scripts.exportDepGraphs.loadClassInfoListFromCache(gav)

fun loadVertexInfo(gav: String) =
    scripts.exportDepGraphs.loadVertexInfo(gav)

In [ ]:
// import common.toJGraph
// import io.github.classgraph.ClassGraph
// import io.github.classgraph.ClassInfoList
// import kotlinx.coroutines.async
// import org.jgrapht.alg.shortestpath.GraphMeasurer
// import org.jgrapht.graph.EdgeReversedGraph
// import org.jgrapht.traverse.BreadthFirstIterator
//
// data class GeneralStat(
//     // val radius: Double,
//     val diameter: Double,
// )
//
// val dispatcher = Dispatchers.IO.limitedParallelism(32)
//
// @kotlinx.coroutines.ExperimentalCoroutinesApi fun _getGeneralStats() =
//     vulnGavToClasses.keys.asSequence().windowed(512, 512, true).flatMapIndexed { batch, w ->
//         runBlocking {
//             System.gc()
//             println("processing batch $batch")
//             w.map {
//                 async(dispatcher) {
//                     val gav = it
//                     val depGraph = loadDepGraph(gav)
//                     val graphMeasurer = GraphMeasurer(depGraph)
//
//                     GeneralStat(
//                         // radius = graphMeasurer.radius,
//                         diameter = graphMeasurer.diameter,
//                     )
//                 }
//             }.toList().awaitAll().toList()  //.filter { it.v != null }
//         }.toList()
//     }.toList()
//
// val stats = _getGeneralStats()
// stats.count()

### how much of the library is dependent on the vuln class?

In [ ]:
import common.toJGraph
import io.github.classgraph.ClassGraph
import io.github.classgraph.ClassInfoList
import kotlinx.coroutines.async
import org.jgrapht.graph.EdgeReversedGraph
import org.jgrapht.traverse.BreadthFirstIterator

data class Res(
    val msg: String? = null,
    val v: Triple<List<Double>, Double, Double>? = null,
    val v2: Double? = null,
)

val dispatcher = Dispatchers.IO.limitedParallelism(32)

@kotlinx.coroutines.ExperimentalCoroutinesApi fun _gavToVulnRatio() =
    vulnCveGavToClasses.asSequence().windowed(512, 512, true).flatMapIndexed { batch, w ->
        runBlocking {
            System.gc()
            println("processing batch $batch")
            w.map {
                async(dispatcher) {
                    val gav = it.key.second
                    val vulnClasses = it.value
                    
                    val depGraph = loadDepGraph(gav)
                    
                    val vulnClassesInJar = vulnClasses.intersect(depGraph.vertexSet())
                    if (vulnClassesInJar.size != vulnClasses.size) {
                        println("vuln class number mismatch ${gav}")
                        println("$vulnClassesInJar")
                        println("$vulnClasses")
                    }

                    val x = vulnClassesInJar.map {
                        val _vulnClasses =
                            BreadthFirstIterator(EdgeReversedGraph(depGraph), it).asSequence().toSet()

                        val vulnRatio = _vulnClasses.size.toDouble() / depGraph.vertexSet().size.toDouble()
                        _vulnClasses to vulnRatio
                    }

                    val vulnClassRatioList = x.map { it.second }
                    val vulnRatioMedian = vulnClassRatioList.median()
                    val vulnRatioAvg = vulnClassRatioList.average()

                    val allVulnClasses = x.flatMap { it.first }.toSet()

                    Res(
                        v = Triple(vulnClassRatioList, vulnRatioMedian, vulnRatioAvg),
                        v2 = allVulnClasses.count().toDouble() / depGraph.vertexSet().count().toDouble()
                    )
                }
            }.toList().awaitAll().toList()  //.filter { it.v != null }
        }.toList()
    }.toList()

val res = _gavToVulnRatio()

// println(res.filter { it.msg != null }.map { it.msg!! }.joinToString("\n"))

In [ ]:
val gavToVulnRatioPerClass = res.map { it.v!! }.toList()
val gavToVulnRatioTotal = res.map { it.v2!! }.toList()
gavToVulnRatioPerClass.count()

In [ ]:
// ([vuln ratio], vuln ratio median, vuln ratio avg)
gavToVulnRatioPerClass.sortedByDescending { abs(it.third - it.second) }.joinToString("\n")

In [ ]:
gavToVulnRatioPerClass.flatMap {
    it.first
}.average()

In [ ]:
gavToVulnRatioPerClass.flatMap {
    it.first
}.median()

In [ ]:
gavToVulnRatioTotal.sorted().joinToString(", ")

In [ ]:
// this is what we care the most
println(gavToVulnRatioTotal.median())
println(gavToVulnRatioTotal.average())

In [ ]:
val data = mapOf(
    "x" to gavToVulnRatioPerClass.map { it.second * 100 } + gavToVulnRatioTotal.map { it * 100 },
    "color" to gavToVulnRatioPerClass.map { "vulnRatioMedianPerClass" } + gavToVulnRatioTotal.map { "vulnRatioMedian" },
)

val p = letsPlot(data) +
        geomHistogram(alpha = .6, binWidth = 5, center = 0, position = positionDodge()) { x = "x"; color = "color"; fill = "color"} +
        xlim(Pair(0, 100)) +
        ggsize(1400, 700)
p

there is little dependency on vuln classes. it means we can hope that if we modularize the libraries, most artifacts (modules) will be non-vulnerable and still usable.